## From Zero Instructions:

1. Install Okareo's Python SDK: &nbsp;&nbsp;  `pip install okareo`  &nbsp;&nbsp;  (just run the cell below)

2. Get your API token from [https://app.okareo.com/](https://app.okareo.com/).  
   (Note: You will need to register first.)

3. Go directly to the **"2. Create your API Token"** link on the landing page in above app. You can skip all other steps.

4. Set the environment variable `OKAREO_API_KEY` to your generated API token.

In [ ]:
%pip install okareo 

In [3]:
# perform a test run using a scenario set from one of the scenario_set notebook examples
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# Callable to be applied to each scenario in the scenario set
def call_model(input: str):
    # call your model being tested here using <input> from the scenario set

    # mock code returnign a random label
    labels = ["returns", "complains", "pricing"]
    import random
    actual = random.choice(labels)

    return actual, {"labels": actual, "confidence": 0.8 }  # return a tuple of (actual, overall model response context)

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="intent_classifier")

# use the scenario id from one of the scenario set notebook examples
scenario_id='ed12729e-25bc-4adc-9a27-c574a460feb7'
test_run_item = model_under_test.run_test(scenario_id=scenario_id, model_invoker=call_model, test_run_name="Intent Classifier Run 3")

# display model level metrics for the test run
print(test_run_item.id)
print(test_run_item.model_metrics.to_dict())

772331a0-dad8-4874-b5a7-9ecfbbea051a
{'weighted_average': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1': 0.3333333333333333, 'accuracy': 0.3333333333333333}, 'scores_by_label': {'complains': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'complaints': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'pricing': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'returns': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}}}


In [3]:
# Retrieve metrics from an earlier test run
import os
from okareo import Okareo

OKAREO_API_KEY = os.environ["OKAREO_API_KEY"]
okareo = Okareo(OKAREO_API_KEY)

# this will return a model if it already exists or create a new one if it doesn't
model_under_test = okareo.register_model(name="intent_classifier")
#test run id from the previous cell output
test_run_item = model_under_test.get_test_run(test_run_id="772331a0-dad8-4874-b5a7-9ecfbbea051a")

# display model level metrics for the test run
print(test_run_item.model_metrics.to_dict())

{'weighted_average': {'precision': 0.3333333333333333, 'recall': 0.3333333333333333, 'f1': 0.3333333333333333, 'accuracy': 0.3333333333333333}, 'scores_by_label': {'complains': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'complaints': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}, 'pricing': {'precision': 1.0, 'recall': 1.0, 'f1': 1.0}, 'returns': {'precision': 0.0, 'recall': 0.0, 'f1': 0.0}}}
